# Cluster the neighborhood data

In [2]:
%store -r Toronto_latlng
Toronto_latlng.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# convert an address into latitude and longitude values and relevant imports

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


# map rendering library

In [15]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

In [16]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [21]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_latlng['latitude'], Toronto_latlng['longitude'], Toronto_latlng['Borough'], Toronto_latlng['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [32]:
Scarborough_data = Toronto_latlng[Toronto_latlng['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [35]:
# create map of Manhattan using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['latitude'], Scarborough_data['longitude'], Scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

In [36]:
CLIENT_ID = 'DGCRD0KX4MNUFQYYHYHEQKPBFYDL5ODAK4HRT2TH0BVOID01' # your Foursquare ID
CLIENT_SECRET = 'MEHE4BBWLMX43PTIEFUEJC3W0YBKYN4ZX0FHTOJ2Q3TC20JL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [37]:
Scarborough_data.loc[0, 'Neighbourhood']

'Rouge,Malvern'

In [38]:
neighborhood_latitude = Scarborough_data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [39]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DGCRD0KX4MNUFQYYHYHEQKPBFYDL5ODAK4HRT2TH0BVOID01&client_secret=MEHE4BBWLMX43PTIEFUEJC3W0YBKYN4ZX0FHTOJ2Q3TC20JL&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [41]:
import requests 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca6067b351e3d25e0aa3bf8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

In [42]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighbourhood'],
                                   latitudes=Scarborough_data['latitude'],
                                   longitudes=Scarborough_data['longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [48]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(88, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [50]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest,Cliffside,Scarborough Village West",2,2,2,2,2,2
"Dorset Park,Scarborough Town Centre,Wexford Heights",7,7,7,7,7,7
"East Birchmount Park,Ionview,Kennedy Park",6,6,6,6,6,6


In [51]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


In [52]:
# one hot encoding
manhattan_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym Pool,Hakka Restaurant,History Museum,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [53]:
manhattan_onehot.shape

(88, 55)

In [54]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym Pool,Hakka Restaurant,History Museum,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.00,0.000000,0.25,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.25,0.0,0.000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.00,0.000000,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.5,0.000000,0.000000,0.000000,0.5,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000000,0.25,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.00,0.000000,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.25,0.0,0.000,0.000000,0.000000
3,Cedarbrae,0.0,0.142857,0.0,0.142857,0.142857,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.000,0.0,0.00,0.000000,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.0,0.000,0.142857,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000000,0.0,0.200000,0.000000,0.000000,0.000000,0.2,0.100000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.100000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.100,0.0,0.00,0.000000,0.00,0.000,0.1,0.000,0.0,0.0,0.000000,0.0,0.1,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.1,0.000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.100000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.100000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.1,0.00,0.000000,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.1,0.0,0.000000,0.100000,0.200000,0.0,0.100,0.000000,0.00,0.0,0.000,0.100000,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.5,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.0,0.00,0.000000,0.00,0.000,0.0,0.000,0.0,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.142857,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.285714,0.00,0.000,0.0,

In [55]:
manhattan_grouped.shape

(16, 55)

In [56]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0         Skating Rink  0.25
1       Breakfast Spot  0.25
2               Lounge  0.25
3       Clothing Store  0.25
4  American Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0
3                Motel   0.0
4     Insurance Office   0.0


----Birch Cliff,Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.14
1       Thai Restaurant  0.14
2                Bakery  0.14
3                  Bank  0.14
4    Athletics & Sports  0.14


----Clairlea,Golden Mile,Oakridge----
                  venue  freq
0              Bus Line   0.2
1                Bakery   0.2
2          Inters

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store,General Entertainment
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Metro Station,Bus Station,Park,Discount Store,College Stadium


In [59]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 1, 2, 1, 1, 1], dtype=int32)

In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels6', kmeans.labels_)

manhattan_merged = Scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

manhattan_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels6,Cluster Labels5,Cluster Labels4,Cluster Labels1,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,0.0,0.0,0.0,0.0,0.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,1.0,1.0,1.0,1.0,1.0,1.0,History Museum,Bar,Moving Target,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Spa,Discount Store,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,3.0,3.0,3.0,3.0,3.0,Coffee Shop,Insurance Office,Korean Restaurant,Vietnamese Restaurant,Gym Pool,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,1.0,1.0,1.0,1.0,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store,General Entertainment


In [78]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['latitude'], manhattan_merged['longitude'], manhattan_merged['Neighbourhood'], manhattan_merged['Cluster Labels6']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='3186cc',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [79]:
manhattan_merged.loc[manhattan_merged['Cluster Labels6'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels6,Cluster Labels5,Cluster Labels4,Cluster Labels1,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,0.0,0.0,0.0,0.0,0.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


In [80]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels6,Cluster Labels5,Cluster Labels4,Cluster Labels1,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,History Museum,Bar,Moving Target,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Spa,Discount Store,Fast Food Restaurant
4,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store,General Entertainment
6,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Chinese Restaurant,Discount Store,Bus Station,Department Store,Convenience Store,Coffee Shop,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store
7,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Metro Station,Bus Station,Park,Discount Store,College Stadium
9,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Furniture / Home Store,Latin American Restaurant,Chinese Restaurant,Bar,Breakfast Spot,Athletics & Sports,Auto Garage
11,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Auto Garage,Bakery,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
12,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Noodle House,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Rental Car Location,Chinese Restaurant,Pharmacy,Italian Restaurant,Coffee Shop


In [81]:
manhattan_merged.loc[manhattan_merged['Cluster Labels6'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels6,Cluster Labels5,Cluster Labels4,Cluster Labels1,ClusterLabels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3.0,3.0,3.0,3.0,3.0,3.0,Coffee Shop,Insurance Office,Korean Restaurant,Vietnamese Restaurant,Gym Pool,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
